In [6]:
#https://pysource.com/2021/08/10/train-mask-r-cnn-for-image-segmentation-online-free-gpu/
%tensorflow_version 2.x
!pip install --upgrade h5py==2.10.0
!wget https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
!unzip Mask_RCNN_basic_1.zip
import sys
sys.path.append("/content/Mask_RCNN/mrcnn")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 14.8 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
--2022-10-31 01:25:24--  https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
Resolving pysource.com (pysource.com)... 188.114.96.3, 188.114.97.3, 2a06:98c1:3120::, ...
Connecting to pysource.com (pysource.com)|188.114.96.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282557 (1.2M) [application/zip]
Saving to: ‘Mask_RCNN_basic_1.zip’

Mask_RCNN_basic_1.z 100%[===================>]   1.22M  --.-KB/s    in 0.02s   

2022-10-31 01:25:24 (57.3 MB/s) - ‘Mask_RCNN_basic_1.zip’ saved [1282557/1282557]

Archive:  Mask_RCNN_basic_1.zip
   creating: Mask_RCNN/
  inflating: Mask_RCNN/.gitignore    
 

AttributeError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: ignored

In [ ]:
# Load Image
img = cv2.imread("/content/drive/MyDrive/CubeSAT/CNN - Detecting Man Made Objects/testimage2.jpg")
from google.colab.patches import cv2_imshow

cv2_imshow(img)
test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/CubeSAT/CNN - Detecting Man Made Objects/demodatasetmodel.h5")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
r = test_model.detect([image])[0]
colors = random_colors(80)


# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        cv2.polylines(img, [cnt], True, colors[i], 2)
        img = draw_mask(img, [cnt], colors[i])

cv2_imshow(img)

In [ ]:
import numpy
imgfiltered = cv2.imread("/content/drive/MyDrive/CubeSAT/CNN - Detecting Man Made Objects/testimage2.jpg")

r["rois"]
contours = []
for i in range(len(r["class_ids"])):
  y1 = r["rois"][i][0]
  x1 = r["rois"][i][1]
  y2 = r["rois"][i][2]
  x2 = r["rois"][i][3]
  print(y1,x1,y2,x2)
  contours.append(numpy.array([[x1, y1], [x1, y2], [x2, y2],[x2,y1]]))
for j in range(len(r['masks'])):
  for i in range(len(r['masks'][j])):
    if r['masks'][j][i][0] == True:
      imgfiltered[j,i] = numpy.array([255,255,255])
    else:
      imgfiltered[j,i] = numpy.array([0,0,0])

    
imggettop = imgfiltered
cv2_imshow(imgfiltered)

In [ ]:
'''
#Same But Bounding
import cv2
import numpy
img = cv2.imread("/content/drive/MyDrive/CubeSAT/CNN - Detecting Man Made Objects/testimage2.jpg")
stencil = numpy.zeros(img.shape).astype(img.dtype)
#contours = r["masks"]
color = [255, 255, 255]

cv2.fillPoly(stencil, contours, color)
cv2.drawContours(img, contours, -1, color=(255, 255, 255), thickness=cv2.FILLED)
result = cv2.bitwise_and(img, stencil)
#cv2_imshow(result)
result[0,0]
img.shape
'''

In [ ]:
#print(imgfiltered.shape[0])
swath_width = int(imgfiltered.shape[0]/5)
stroke_width = 5
nth = 1
while((imgfiltered.shape[0] - swath_width*nth)>swath_width):
  for i in range(imgfiltered.shape[1]):
    for r in range(stroke_width):
      imgfiltered[swath_width*nth+r,i] = numpy.array([255,0,0])
  nth+=1
cv2_imshow(imgfiltered)
nth

In [ ]:
#Find Best Swath
topsum = 0
bestnth = 100
for i in range(nth):
  this_topsum = 0
  for y in range(swath_width):
    for x in range(imgfiltered.shape[1]):
      if imgfiltered[i*swath_width+y,x][1] == 255:
        this_topsum +=1
  if this_topsum>topsum:
    topsum = this_topsum
    bestnth = nth
print(bestnth)